# Compartments

In [5]:
r_enz = 'HindIII'
rep   = 'T0'
reso = 100000

In [4]:
from pytadbit.parsers.hic_parser import load_hic_data_from_reads
from cPickle import load

In [ ]:
hic_data = load_hic_data_from_reads(
    'results/fragment/{0}_{1}/03_filtering/valid_reads12_{0}_{1}.tsv'.format(r_enz, rep),
    100000)